Bril
llvm
xdsl

nanopass

vectorization

mini python 

https://github.com/IUCompilerCourse/Essentials-of-Compilation
https://github.com/IUCompilerCourse/python-student-support-code

https://github.com/IUCompilerCourse/python-student-support-code/blob/main/interp_x86/parser_x86.py


https://github.com/cslarsen/minijit
https://github.com/zserge/tinylangs/blob/main/asm.py
https://github.com/benhoyt/pyast64  https://benhoyt.com/writings/pyast64/
https://github.com/Maratyszcza/PeachPy
https://github.com/ssloy/tinycompiler

https://news.ycombinator.com/item?id=14644576

https://gitlab.com/tsoding/porth https://www.youtube.com/watch?v=QlKSCn-Al6Q&ab_channel=TsodingDaily


Some mix of meta and internalized

In [17]:
from kdrag.all import *
BV64 = smt.BitVecSort(64)
getarg = smt.Function("getarg", smt.IntSort(), BV64)

def remove_complex_operands(e : smt.ExprRef):
    todo = [e]
    prog = []
    while todo:
        e = todo.pop()
        prog.append(e)
        if isinstance(e, smt.BitVecNumRef):
            continue
        if smt.is_app(e):
            if e.decl() != getarg:
                assert e.num_args() <= 2
                todo.extend(e.children())
        else:
            raise ValueError("Expected an application, got: ", e)
    return list(reversed(prog))
prog = getarg(0) + 42 - getarg(1)
prog1 = remove_complex_operands(prog)

In [ ]:
def remove_complex_correct(prog0, prog1):
    # There's nothing much to say here.
    pass

In [ ]:
movq = smt.Function("movq", BV64, BV64)
subq = smt.Function("subq", BV64, BV64, BV64)
addq = smt.Function("addq", BV64, BV64, BV64)
Reg = kd.Enum("Reg", ["RAX", "RBX", "RCX", "RDX", "RSI", "RDI", "RSP", "RBP", "R8", "R9", "R10", "R11", "R12", "R13", "R14", "R15"])
Insn = kd.Enum("Insn", ["movq", "addq", "subq"])

def instruction_select(prog : list[smt.ExprRef]):
    res = []

    for expr in prog:
        if isinstance(expr, smt.BitVecNumRef):
            continue
        else:
            match expr.decl().name():
                case "getarg":
                    i = expr.arg(0)
                    assert isinstance(i, smt.IntNumRef)
                    reg = [Reg.RDI, Reg.RSI, Reg.RDX, Reg.RCX, Reg.R8, Reg.R9][i.as_long()]
                    res += [(movq, reg, expr)]
                case "bvadd":
                    arg1,arg2 = expr.children()
                    res += [(movq, arg1, Reg.RAX),
                            (addq, arg2, Reg.RAX),
                            (movq, Reg.RAX, expr)]
                case "bvsub":
                    arg1,arg2 = expr.children()
                    res += [(movq, arg1, Reg.RAX),
                            (subq, arg2, Reg.RAX),
                            (movq, Reg.RAX, expr)]
                
                case _:
                    raise ValueError("Unexpected expression in instruction selection: ", expr)
    return res


prog2 = instruction_select(prog1)
prog2 

[(movq, RDI, getarg(0)),
 (movq, getarg(0), RAX),
 (addq, 42, RAX),
 (movq, RAX, getarg(0) + 42),
 (movq, RSI, getarg(1)),
 (movq, getarg(0) + 42, RAX),
 (subq, getarg(1), RAX),
 (movq, RAX, getarg(0) + 42 - getarg(1))]

In [ ]:
rsp = smt.BitVec("rsp", 64)

def assign_home(prog):
    res = []
    stack = {}
    for stmt in prog:
        match stmt:
            case (op, arg1, arg2):
                print(f"{op} {arg1} {arg2}")
                if arg1.sort() == BV64:
                    if arg1 not in stack:
                        stack[arg1] = rsp + len(stack) * 8
                res.append((op, stack[arg1], stack[arg2]))
            case _:
                raise ValueError("Unexpected statement in assignment: ", stmt)
    return res
assign_home(prog2)

movq RDI getarg(0)
movq getarg(0) RAX
addq 42 RAX
movq RAX getarg(0) + 42
movq RSI getarg(1)
movq getarg(0) + 42 RAX
subq getarg(1) RAX
movq RAX getarg(0) + 42 - getarg(1)


In [ ]:
    env = {}
    regfile = smt.Const("undefined", Regfile)
regfile[[RDI, RDX][i.as_long()]]
Regfile = smt.ArraySort(Reg, BV64)

mov_to_rax = smt.Function("movq", BV64, Regfile, Regfile)
read_rax = smt.Function("read_rax", Regfile, BV64)
add_rax = smt.Function("add_rax", BV64, Regfile, Regfile)




In [ ]:
Stmt = smt.ArraySort(State, State)
RR = smt.Function("RR", Insn, Reg,   Reg,       State, State) # Reg Reg insn
smt.Function("MR", Insn, Reg, BV64,  Reg,  State, State) # Mem Reg insn
smt.Function("IR", Insn, BV64,   Reg,      State, State) # Imm Reg insn
smt.Function("RM", Insn, Reg,  Reg, BV64,  State, State) # Reg Reg insn

def from_tup(insn):
    insn, op1, op2 = insn
    s1,s2 = op1.sort(), op2.sort()
    if s1 == Reg and s2 == Reg:
        return RR(insn, op1, op2)
    elif s1 == Reg and s2 == BV64:
        return MR(insn, op1, op2)
    elif s1 == BV64 and s2 == Reg:
        return RM(insn, op1, op2)



In [ ]:
Stack = smt.ArraySort(smt.BitVecSort(8), BV64)
Reg = kd.Enum("Reg", ["RAX", "RBX", "RCX", "RDX", "RSI", "RDI", "RSP", "RBP"])
Regfile = smt.ArraySort(Reg, BV64)
movqrr = smt.Function("movqrr", Reg, Reg,  Regfile, Regfile)
movqlr = smt.Function("movqlr", BV64, Reg, Regfile, Regfile)
movqrl = smt.Function("movqrl", BV64, Reg, Regfile, Regfile)
movqsr = smt.Function("movqsr", Reg, Stack, Regfile, Regfile)
movqrs = smt.Function("movqrs", Stack, Reg, Regfile, Stack)

State = kd.Struct("x86State", ("regs", Regfile), ("stack", Stack))


TypeError: Enum() takes 2 positional arguments but 9 were given

target interpreter



In [ ]:
%%file /tmp/test.S

.globl mytest
mytest:
    mov $42, %rax
    ret

In [ ]:
!gcc -c -fPIC -o /tmp/test.o /tmp/test.S && ld -shared -o /tmp/test2.so /tmp/test.o && objdump -d /tmp/test.so

In [ ]:
import cffi

ffi = cffi.FFI()
ffi.cdef("""uint64_t mytest();""")
lib = ffi.dlopen("/tmp/test2.so")
dir(lib)
print(lib.mytest())
del lib

def 